# Chapter 3: Variational Autoencoders

## Building a Variational Autoencoder

A _variational autoencoder_ (VAE) is an autoencoder that encodes inputs into a _normal distribution_ that can be used to sample the latent space. The normal distribution in one dimension is given by

$$ f \left( x \,|\, \mu, \sigma \right) = \frac{1}{\sqrt{2\pi\sigma^2}} \, \exp \left( - \frac{(x - \mu)^2}{2\sigma^2} \right) $$

Its parameters are the _mean_, $\mu$, and the standard deviation, $\sigma$. The _standard normal distribution_ has a mean of zero and a standard deviation of one. The multidimensional normal distribution is given by

$$ f \left( \mathbf{x} \,|\, \mu,\sigma \right) =  \frac{\exp\left( -\frac{1}{2} (\mathbf{x} - \mu)^T \Sigma^{-1} (\mathbf{x} - \mu) \right)}{\sqrt{(2\pi)^k|\Sigma|}} $$

where $k$ is the number of dimensions, $\mu$ is the mean vector, and $\Sigma$ is the _covariance matrix_, which is given by

$$ \Sigma_{ij} = \left\{ \begin{matrix} \sigma_i^2 && \text{if}\;i = j \\ \rho_{ij} \sigma_i\sigma_j && \text{otherwise} \end{matrix} \right. $$

where $\rho_{ij}$ is the correlation between the two dimensions. Variational autoencoders assume that $\rho = 0$, i.e. that the dimensions in the latent space have no correlation. This means $\Sigma$ is a diagonal matrix whose values are th variance in each dimension. When training a VAE, we use the logarithm of the variance since that can fall anywhere on the interval $(-\infty,\infty)$.

### Loss Function

In addition to using MSE for the reconstruction loss, we also introduce the _Kullback Leibler (KL) divergence_, a way to measure how different a probability distribution is from another. In general it is given by

$$ \begin{align} D_{KL}\left(P\,||\,Q\right) & = -\sum\limits_{x\in X} p(x)\log q(x) + \sum\limits_{x\in X} p(x)\log p(x) \\ & = H(P,Q) - H(P) \end{align} $$

where $H(P,Q)$ is the _cross-entropy_ between $P$ and $Q$, and $H(P)$ is the entropy of P. In our case, we are measuring the KL divergence between the model and the standard normal distribution. In this special case, the KL divergence has the closed form

$$ D_{KL}\left[N(\mu,\sigma)\,||\,N(0,1)\right] = \frac{1}{2}\sum\left(1+\log\left(\sigma^2\right)-\mu^2-\sigma^2\right) $$

### Using a VAE to Reconstruct Digits

Below is an example implementation of a VAE, we will use this to reconstruct handwritten digits from the MNIST dataset. Later on, we will use it to reconstruct faces from the CelebA dataset.

In [0]:
# Mount Google drive to save the weights after training.
from google.colab import drive

drive.mount('/content/gdrive/')

In [0]:
# Setup, this cell should be run every time.

from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization,
                                     LeakyReLU, Dropout, Flatten, Dense,
                                     Lambda, Reshape, Conv2DTranspose,
                                     Activation)
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import tensorflow as tf
from tensorflow.keras.losses import KLDivergence, MeanSquaredError


def sampling(args):
  """Sample an encoding from the learned distribution."""
  mu, log_var = args
  return mu + K.exp(log_var / 2) * K.random_normal(shape=K.shape(mu))


def step_decay_schedule(initial_lr, decay_factor=0.5, step_size=1):
  """Create a LearningRateScheduler callback to decay the learning rate during training."""
  def schedule(epoch):
    return initial_lr * (decay_factor ** np.floor(epoch/step_size))
  return LearningRateScheduler(schedule)


class VariationalAutoencoder():
  """Implements a Varational Autoencoder."""

  def __init__(self, input_shape, encoder_conv_filters,
               encoder_conv_kernel_size, encoder_conv_strides,
               decoder_conv_filters, decoder_conv_kernel_size,
               decoder_conv_strides, z_dim, use_batch_normalization=False,
               use_dropout=False, dropout_rate=0.25):
    encoder_input = Input(shape=input_shape, name='encoder_input')
    x = encoder_input
    for i in range(len(encoder_conv_kernel_size)):
      x = Conv2D(filters=encoder_conv_filters[i],
                 kernel_size=encoder_conv_kernel_size[i],
                 strides=encoder_conv_strides[i], padding='same',
                 name='encoder_conv_{}'.format(i + 1))(x)
      if use_batch_normalization:
        x = BatchNormalization()(x)
      x = LeakyReLU()(x)
      if use_dropout:
        x = Dropout(rate=dropout_rate)(x)
    shape_before_flattening = K.int_shape(x)[1:]
    x = Flatten()(x)
    self.mu = Dense(z_dim, name='mu')(x)
    self.log_var = Dense(z_dim, name='log_var')(x)
    self.encoder_mu_log_var = Model(encoder_input, (self.mu, self.log_var))
    encoder_output = Lambda(sampling,
                            name='encoder_output')([self.mu, self.log_var])
    self.encoder = Model(encoder_input, encoder_output)

    decoder_input = Input(shape=(z_dim,), name='decoder_input')
    x = Dense(np.prod(shape_before_flattening))(decoder_input)
    x = Reshape(shape_before_flattening)(x)
    for i in range(len(decoder_conv_kernel_size)):
      x = Conv2DTranspose(filters=decoder_conv_filters[i],
                          kernel_size=decoder_conv_kernel_size[i],
                          strides=decoder_conv_strides[i], padding='same',
                          name='decoder_conv_t_{}'.format(i + 1))(x)
      if i < len(decoder_conv_kernel_size) - 1:
        if use_batch_normalization:
          x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        if use_dropout:
          x = Dropout(rate=dropout_rate)(x)
      else:
        x = Activation('sigmoid')(x)
    decoder_output = x
    self.decoder = Model(decoder_input, decoder_output)
    self.model = Model(encoder_input, self.decoder(encoder_output))
    self.compiled = False
    self.learning_rate = None

  def compile(self, learning_rate, r_loss_factor):
    """Compile the model."""
    self.learning_rate = learning_rate
    if self.compiled:
      return
    opt = Adam(lr=learning_rate)

    mse = MeanSquaredError()
    kl_divergence = KLDivergence()

    def loss(y_act, y_pred):
      return mse(y_act, y_pred) + kl_divergence(y_act, y_pred)
    
    self.model.compile(opt, loss=loss, metrics=[mse, kl_divergence],
                       experimental_run_tf_function=False)
    self.compiled = True

  def fit(self, X, y, batch_size, epochs, checkpoint_path=None, lr_decay=1):
    """Train the model."""
    if not self.compiled:
      raise Exception('Model not compiled')
    lr_sched = step_decay_schedule(initial_lr=self.learning_rate,
                                   decay_factor=lr_decay, step_size=1)
    callbacks = [lr_sched]
    if checkpoint_path:
      callbacks.append(ModelCheckpoint(filepath=checkpoint_path, verbose=1,
                                       save_weights_only=True))
    self.model.fit(X, y, batch_size=batch_size, epochs=epochs, shuffle=True,
                   callbacks=callbacks)

  def load(self, checkpoint_path):
    """Load the model weights from the saved checkpoint file."""
    self.model.load_weights(checkpoint_path)

In [28]:
mnist_vae = VariationalAutoencoder(input_shape=(28, 28, 1),
                                   encoder_conv_filters=(32, 64, 64, 64),
                                   encoder_conv_kernel_size=(3, 3, 3, 3),
                                   encoder_conv_strides=(1, 2, 2, 1),
                                   decoder_conv_filters=(64, 64, 32, 1),
                                   decoder_conv_kernel_size=(3, 3, 3, 3),
                                   decoder_conv_strides=(1, 2, 2, 1),
                                   z_dim=2)
mnist_vae.model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_14 (LeakyReLU)      (None, 28, 28, 32)   0           encoder_conv_1[0][0]             
__________________________________________________________________________________________________
encoder_conv_2 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu_14[0][0]             
___________________________________________________________________________________________

In [0]:
# Load the MNIST dataset.

from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
# Reshape these to be 4D tensors and scale the pixel values to [0, 1].

X_train = X_train.reshape(X_train.shape + (1,)) / 255.0
X_test = X_test.reshape(X_test.shape + (1,)) / 255.0

In [0]:
# The path to save the model weights to in Drive.

checkpoint_path = '/content/gdrive/My Drive/gdl_models/mnist_vae.hdf5'
mnist_vae.compile(learning_rate=0.0005, r_loss_factor=1000)

Now we will train the model for 200 training epochs.

In [0]:
mnist_vae.fit(X_train, X_train, batch_size=32, epochs=200,
              checkpoint_path=checkpoint_path)

Epoch 1/200
1871/1875 [============================>.] - ETA: 0s - loss: 0.0876 - mean_squared_error: 0.0713 - kullback_leibler_divergence: 0.0163
Epoch 00001: saving model to /content/gdrive/My Drive/gdl_models/mnist_vae.hdf5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0876 - mean_squared_error: 0.0713 - kullback_leibler_divergence: 0.0163 - lr: 5.0000e-04
Epoch 2/200
1866/1875 [============================>.] - ETA: 0s - loss: 0.0831 - mean_squared_error: 0.0686 - kullback_leibler_divergence: 0.0145
Epoch 00002: saving model to /content/gdrive/My Drive/gdl_models/mnist_vae.hdf5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0831 - mean_squared_error: 0.0686 - kullback_leibler_divergence: 0.0145 - lr: 5.0000e-04
Epoch 3/200
1874/1875 [============================>.] - ETA: 0s - loss: 0.0807 - mean_squared_error: 0.0673 - kullback_leibler_divergence: 0.0135
Epoch 00003: saving model to /content/gdrive/My Drive/gdl_models/mnist_vae.hdf5
1875/